## Setup

In [ ]:

!pip install -q transformers accelerate peft bitsandbytes


In [ ]:
# verify if GPU is there
import torch
torch.cuda.is_available()

In [ ]:
# Check the GPU
!nvidia-smi

## Demonstration LORA

### Setup

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "microsoft/phi-2"
#model_name = "distilgpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype=torch.float16,
    device_map="auto"
)



In [ ]:
system_message = "You are a helpful Assistant"
q1="how can I change my password?"
q2 = "I keep procrastinating and lack discipline. What would Chanakya advise?"
q3="what to do if I forgoet my password?"
q4="Please help me to reset my password"

In [ ]:

def ask_genai_course_advisor(question, model, tokenizer, max_new_tokens=60):
    """
    Ask the fine-tuned model a question.
    """

    prompt = f"""{system_message}.
Question: {question}
Answer:"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    model.eval()
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            repetition_penalty=1.3,
            no_repeat_ngram_size=3,
            temperature=0.0,
            do_sample=False
        )

    generated = out[0][inputs["input_ids"].shape[-1]:]
    return tokenizer.decode(generated, skip_special_tokens=True).strip()


### Before

In [ ]:
# inputs are already calculated in "BEFORE block"
# Model is in memory

answer = ask_genai_course_advisor(
    q2,
    model,
    tokenizer
)

print(answer)


### Training

In [ ]:
from datasets import Dataset

# Training data split into question + answer (explicit structure)
train_data = [
    {
        "question": "I feel distracted all the time. How can I improve my focus?",
        "answer": "Chanakya teaches that the mind follows discipline. Reduce unnecessary desires, set one clear goal, and eliminate distractions without mercy."
    },
    {
        "question": "How did Chanakya view focus and self-control?",
        "answer": "According to Chanakya, focus is born from self-control. A person who cannot govern their senses cannot govern their destiny."
    },
    {
        "question": "What is Chanakya’s advice for mental clarity?",
        "answer": "Chanakya advised waking early, maintaining routine, and avoiding indulgence. A structured life creates a focused mind."
    },
    {
        "question": "How can I train my mind to stay focused?",
        "answer": "Chanakya believed the mind must be trained like a warrior. Daily discipline, limited comforts, and repeated practice sharpen focus."
    },
    {
        "question": "Why do I lose focus easily according to Chanakya?",
        "answer": "Chanakya taught that lack of purpose weakens focus. When the goal is unclear, the mind wanders without direction."
    },
    {
        "question": "What habits destroy focus?",
        "answer": "Chanakya warned against excess sleep, uncontrolled speech, and constant pleasure-seeking, as these slowly destroy concentration."
    },
    {
        "question": "How does one build strong concentration?",
        "answer": "Chanakya advised doing one task at a time with full attention. Divided effort leads to divided results."
    }
]





dataset = Dataset.from_list(train_data)

def tokenize(batch):
    input_ids = []
    labels = []
    attention_masks = []

    for q, a in zip(batch["question"], batch["answer"]):
        prompt = f"{system_message}.\nQuestion: {q}\nAnswer:"
        answer = f" {a}"

        # Tokenize prompt + answer together
        full = tokenizer(
            prompt + answer,
            truncation=True,
            padding="max_length",
            max_length=128
        )

        # Tokenize prompt alone (to know where answer starts)
        prompt_tokens = tokenizer(prompt, truncation=True, max_length=128)

        label = [-100] * len(full["input_ids"])

        answer_start = len(prompt_tokens["input_ids"])
        label[answer_start:] = full["input_ids"][answer_start:]

        input_ids.append(full["input_ids"])
        labels.append(label)
        attention_masks.append(full["attention_mask"])

    return {
        "input_ids": input_ids,
        "labels": labels,
        "attention_mask": attention_masks
    }

dataset = dataset.map(tokenize, batched=True, remove_columns=dataset.column_names)
dataset.set_format("torch")


In [ ]:
# LORA
from peft import LoraConfig, get_peft_model

if model_name == "distilgpt2":
  lora_config = LoraConfig(
      r=8,
      lora_alpha=16,
      target_modules=["c_attn", "c_proj"],
      lora_dropout=0.05,
      task_type="CAUSAL_LM"
  )
else:
  lora_config = LoraConfig(
      r=8,
      lora_alpha=16,
      target_modules=["q_proj", "v_proj"],
      lora_dropout=0.05,
      task_type="CAUSAL_LM"
  )

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()



In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./demo",
    per_device_train_batch_size=1,
    num_train_epochs=1,
    max_steps=80,
    logging_steps=1,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)

trainer.train()

model.eval()
model.print_trainable_parameters()



In [ ]:
# Save and reopen
"""
model.save_pretrained("./demo_updated")
tokenizer.save_pretrained("./demo_updated")

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)
model = get_peft_model(model, lora_config)
model.load_adapter("./demo_updated")
"""


In [ ]:
# alternate way of directly updating the model in memory
model = model.merge_and_unload()
model.eval()

### After

In [ ]:
# inputs are already calculated in "BEFORE block"
# Model is in memory
#question= "Is there any interview support?"
answer = ask_genai_course_advisor(
    q2,
    model,
    tokenizer
)

print(answer)
